# Plaidbot Training

This notebook is for training a plaidbot model

Change runtime to use a GPU

## Setup

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r '/content/drive/MyDrive/Colab Notebooks/Plaidbot/v2/src' .

In [ ]:
# Imports
from datetime import datetime

# Options
from options.model_options import ModelOptions
from options.prepro_options import PreproOptions

# Scripts
from src.training.pick_users.run_pick_users import run_pick_users
from src.training.select_data.run_select_data import run_select_data
from src.training.train.run_training import run_training
from src.training.train.run_evaluation import run_evaluation

## Options

In [ ]:
prepro_opts = PreproOptions()

# File path options
prepro_opts.user_filename = 'users.json' # File from slack where user info is stored 
prepro_opts.message_folder = '/content/drive/MyDrive/Colab Notebooks/Plaidbot/messages' # Folder containing slack message folders
prepro_opts.selected_folders = [ 
    'general',
    # add desired folders...
]

# Filtering Options
prepro_opts.min_date: datetime  = datetime(2018, 1, 1) # Earliest message date
prepro_opts.min_num_words = 3 # Minimum number of words in a message
prepro_opts.max_messages = 100000 # Maximum number of messages to use for training and testing

In [ ]:
model_opts = ModelOptions()
model_opts.bert_model_name = 'distilbert-base-uncased' # Bert model name
model_opts.max_len = 150 # Max characters per message
model_opts.val_size = 0.2 # Proportion of messages used for validation
model_opts.num_epochs = 2 # Number of training epochs
model_opts.batch_size = 8 # Batch size for data loader
model_opts.device = 'cuda:0' # Device used for training. Use 'cuda:0' for training, 'cpu' for deploying
model_opts.learning_rate = 2e-5 # Learning rate of optimizer

model_opts.saved_model_name = 'username/model-name' # Saved model name for HuggingFace
model_opts.auth_token = 'hugging-face-auth-token-goes-here' # HuggingFace Access token

## Pick users and Data

In [ ]:
a, b = run_pick_users(prepro_opts)

In [ ]:
prepro_opts.user_id_int_dict = a
model_opts.user_int_name_dict = b

In [ ]:
train_messages, test_messages = run_select_data(prepro_opts)

## Train and Evaluate

In [ ]:
model = run_training(train_messages, model_opts)

In [ ]:
run_evaluation(model, test_messages, model_opts)

## Save the model

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Maybe I should also save the tokenizer?
base_model = model.get_inner_model().get_base_model()
base_model.push_to_hub(model_opts.saved_model_name)

## Reload Model and Predict

In [ ]:
from src.web.web_dependencies import WebDependencies
web_deps = WebDependencies(model_opts)

In [ ]:
sample_text = 'This is a test!'
prediction = web_deps.prediction_handler.handle(sample_text)
print(prediction)